In [22]:
%%time

import nltk
import numpy
import time
import random
import unidecode
import string
import heapq

import pandas as pd

from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize, WordNetLemmatizer
from numba import njit, jit, cuda
import numpy as np

nltk.download('stopwords')
nltk.download('wordnet')

%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
CPU times: user 1.1 ms, sys: 3 µs, total: 1.1 ms
Wall time: 893 µs


[nltk_data] Downloading package stopwords to /home/tmtoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tmtoon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
AMOUNT_OF_TOPICS = 100

# top % of most common words are removed
TOP_K_WORD_REMOVAL = 0.001

# https://stats.stackexchange.com/questions/59684/what-are-typical-values-to-use-for-alpha-and-beta-in-latent-dirichlet-allocation/130876
# High alpha: many topics per document [low : 1 : high]
ALPHA            = 5
# High beta: topic has many different words [0 - 1]
BETA             = 0.1



In [24]:
%%time

df = pd.read_csv("news_dataset.csv")
#df = df[:25000]
df = df[df['content'].notna()]

df["documents"] = df["content"]

CPU times: user 4.49 s, sys: 96.1 ms, total: 4.59 s
Wall time: 4.58 s


In [25]:
%%time

# remove accents, special characters and lowercase
# https://stackoverflow.com/questions/37926248/how-to-remove-accents-from-values-in-columns
df.content = df.content.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace('[^a-z ]', '')
print(df.content.head())

0    washington eeaaaaeeeace    congressional repub...
1    after the bullet shells get counted the blood ...
2    when walt disneys bambi opened in  critics pra...
3    death may be the great equalizer but it isnt n...
4    seoul south korea     north koreas leader kim ...
Name: content, dtype: object
CPU times: user 10.2 s, sys: 92.1 ms, total: 10.3 s
Wall time: 10.3 s


In [26]:
%%time
print(len(df['content'][0]))
lemmatizer = WordNetLemmatizer()
def remove_singles(content):
    d = dict()
    for word in content.split(' '):
        word = lemmatizer.lemmatize(word)
        if word in d:
            d[word] += 1
        else:
            d[word] = 1
    
    new = ""
    for word, val in d.items():
        if val > 1 and word != '':
            new += (word + " ") * val
            
    return new

df['content'] = df['content'].apply(lambda content: remove_singles(content))
print(len(df['content'][0]))

5498
3316
CPU times: user 5min 42s, sys: 14.8 ms, total: 5min 42s
Wall time: 5min 42s


In [27]:
%%time

# remove stop words and single letters
# https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe
stopwords_list = stopwords.words('english')
stopwords_list.extend(list(string.ascii_lowercase))
#pat = r'\b(?:{})\b'.format('|'.join(stopwords_list))
#df['content'] = df['content'].str.replace(pat, '')
#print(df.content.head())

CPU times: user 988 µs, sys: 3 µs, total: 991 µs
Wall time: 676 µs


In [28]:
%%time


fdict = dict()

if TOP_K_WORD_REMOVAL > 0:

    # remove common words
    for i in df.content.str.split(' '):
        i = list(filter(lambda x:x!="" , i))
        for word in i:
            if word != '':
                if word in fdict:
                    fdict[word] += 1
                else:
                    fdict[word] = 1

CPU times: user 28.5 s, sys: 1.07 s, total: 29.6 s
Wall time: 29.6 s


In [29]:
%%time

length = int(TOP_K_WORD_REMOVAL * len(fdict))
top_k_words = heapq.nlargest(length, fdict, key=fdict.get)

top_k_words.extend(stopwords_list)

pat = r'\b(?:{})\b'.format('|'.join(top_k_words))
print(pat)
df['content'] = df['content'].str.replace(pat, '')
print(df.content.head())

\b(?:the|a|to|of|and|in|that|it|for|is|on|he|wa|with|said|his|at|i|have|by|but|be|trump|from|are|not|ha|this|an|they|who|about|we|you|or|their|were|more|had|she|her|one|will|would|people|been|there|year|u|state|when|if|which|what|say|all|new|out|so|time|up|president|after|like|than|also|can|s|clinton|no|some|just|him|other|do|our|them|my|into|could|over|republican|mr|how|american|because|first|even|now|two|woman|day|country|get|many|most|make|percent|campaign|last|those|government|way|house|only|told|me|think|company|these|right|police|dont|against|white|where|news|any|know|going|did|before|group|while|want|may|i|me|my|myself|we|our|ours|ourselves|you|you're|you've|you'll|you'd|your|yours|yourself|yourselves|he|him|his|himself|she|she's|her|hers|herself|it|it's|its|itself|they|them|their|theirs|themselves|what|which|who|whom|this|that|that'll|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|wit

In [30]:
%%time

# remove multiple spaces in a
df['content'] = df['content'].str.replace(r'\s+', ' ')
print(df.content.head())

0    congressional congressional congressional come...
1     blood blood blood window window see see see c...
2     walt walt walt disney disney disney disney di...
3    death death death death death death death deat...
4    south south south south south south south kore...
Name: content, dtype: object
CPU times: user 7.81 s, sys: 0 ns, total: 7.81 s
Wall time: 7.81 s


In [31]:
%%time

vocab_set = set()

def compute_dict(input):
    result = dict()

    for word in input.split(' '):
        if word != '':
            vocab_set.add(word)

df['content'].apply(lambda content: compute_dict(content))
vocab = list(vocab_set)

tmp = {}
count = 0
for word in vocab:
    tmp[word] = count
    count += 1
vocab = tmp

CPU times: user 4.08 s, sys: 0 ns, total: 4.08 s
Wall time: 4.08 s


In [32]:
%%time

#Toevoegen van extra column met index ineens
def remove_dupes(input):
    output = input.split(' ')
    output = list(filter(lambda x:x!="" , output))
    for i, value in enumerate(output):
        output[i] = vocab[value]
    return np.array(output)
df['content_list'] = df['content'].apply(lambda content: remove_dupes(content))

CPU times: user 7.67 s, sys: 0 ns, total: 7.67 s
Wall time: 7.67 s


In [33]:
%%time

numberOfTopics = AMOUNT_OF_TOPICS
topics = [i for i in range(0, numberOfTopics)]

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 12.9 µs


In [34]:
%%time

wordToTopic = numpy.zeros((len(vocab), numberOfTopics), dtype=numpy.int64)
documentToTopic = numpy.zeros((len(df.content), numberOfTopics), dtype=numpy.int64)
topicAssignment = numpy.zeros(len(df.content_list), dtype=numpy.ndarray)
words_in_topic = numpy.zeros(numberOfTopics, dtype=numpy.int64)
words_in_doc = [0 for i in range(len(df.content))]
for i, content in enumerate(df.content_list):
    tmp = np.zeros(len(content), dtype=numpy.int64)
    for word_index, word in enumerate(content):
        topic = topics[int(np.random.rand()*numberOfTopics)]
        tmp[word_index] = topic
        wordToTopic[word, topic] += 1
        documentToTopic[i, topic] += 1
        words_in_topic[topic] += 1
        words_in_doc[i] += 1
    topicAssignment[i] = tmp

CPU times: user 1min 1s, sys: 117 ms, total: 1min 1s
Wall time: 1min 1s


In [35]:
%%time

alpha = ALPHA
beta = BETA

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


In [36]:
%%time

@njit
def calc_fast(rand_val, word_index, word, TA_d, doc_index, wordToTopic, words_in_topic, total_unique_word_b, document_words, WID):
    
        document_words[TA_d[word_index]] -= 1               
        wordToTopic[word, TA_d[word_index]] -= 1
        words_in_topic[TA_d[word_index]] -= 1

        new_topic = (np.multiply(np.divide((wordToTopic[word] + beta), 
                                                       (words_in_topic + total_unique_word_b)), 
                                             np.divide((document_words + alpha), 
                                                       (WID))))

        new_topic = np.divide(new_topic, np.sum(new_topic))
        new_topic = np.cumsum(new_topic)
        topic = 0
        for i in range(0, numberOfTopics):
            if rand_val[word_index] < new_topic[i]:
                topic = i
                break

        new_topic = topic
        TA_d[word_index] = new_topic
        document_words[new_topic] += 1
        wordToTopic[word, new_topic] += 1
        words_in_topic[new_topic] += 1
    
def gibs_it_sanic(i):
    total_unique_word_b = len(vocab) * beta
    topic_count_a = numberOfTopics * alpha
    for iteration in range(i):
        print("Iteration:", iteration + 1)
        
        for doc_index, word_list in enumerate(df.content_list):
            document_words = documentToTopic[doc_index]
            WID = words_in_doc[doc_index] + topic_count_a
            TA_d = topicAssignment[doc_index]
            
            rand_val = np.random.random(len(word_list))
            for word_index, word in enumerate(word_list):
                calc_fast(rand_val, word_index, word, TA_d, doc_index, wordToTopic, words_in_topic, total_unique_word_b, document_words, WID)

                


CPU times: user 353 µs, sys: 0 ns, total: 353 µs
Wall time: 357 µs


In [37]:
%%time
gibs_it_sanic(25)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
CPU times: user 19min 48s, sys: 605 ms, total: 19min 49s
Wall time: 19min 45s


In [38]:
# %lprun -f gibs_it_sanic gibs_it_sanic(10)

In [39]:
t2w = pd.DataFrame(wordToTopic).T
t2w.columns = vocab

top_words_per_topic = [
    (t2w.iloc[k][t2w.iloc[k] > 0.001].sort_values(ascending=False).index.values.tolist()[:9])
    for k in topics
]


from tabulate import tabulate
print(tabulate(top_words_per_topic))

------------  -------------  --------------  ------------  -------------  --------------  ----------  --------------  --------------
water         area           town            home          city           resident        storm       florida         coast
fox           kelly          allegation      lawsuit       sexual         complaint       host        harassment      crew
north         south          defense         official      carolina       korea           dog         korean          kim
found         animal         death           heart         age            life            blood       africa          teen
car           vehicle        model           driver        uber           safety          technology  road            tesla
million       debt           loan            dnc           bond           chair           puerto      hamilton        crisis
set           point          open            second        williams       break           final       third           next
lif

In [40]:
t2d = pd.DataFrame(documentToTopic.T)

top_docs_per_topic = [
    (t2d.iloc[k][t2d.iloc[k] > 0.001].sort_values(ascending=False).index.values.tolist()[:5])
    for k in topics
]


from tabulate import tabulate
print(tabulate(top_docs_per_topic))

------  ------  ------  ------  ------
 15664  135271   64454    3201  127770
101692  129245  141315  129278    2248
137002   70174   69914   69896   32807
 55205  113970   50716   77505  134915
122565   69028  107571  128723   54796
121259   26829   70605  128507  121440
103633  101680  101716  102403  100172
 53790   53787   66598   53528   67591
133097  136170   56268   13680   55656
 51008   73459   76470  128345  126663
138339    3133  129048    6617  131397
 68818   68885  126132  131595     260
126572   51165   50169   56460   55662
126290   56268   11505   34280  128709
 70605   36905   68420   68525   68442
  7063  103601   56503   39004   77275
128405    5184  128070    6390  136581
 71781   73143   53226   71754   73742
  6407   22071   96719  110328  101367
 30798   28739   16846    8796   25008
 53551   53515    4659   71754  136572
  3467   68004   55706    2436  100543
126218   71079  140283    1523   96711
136568   50465   55352   34288    2788
  7111   67564   53718   

In [41]:
%%time

docs = len(df.content_list)
score = 0
for tops in top_words_per_topic:
    for i in range(0, len(tops)-2):
        w_i = tops[i]
        w_j = tops[i+1]
        
        c1 = 0
        c2 = 0
        c3 = 0
        for item in df.content_list:
            tmp = 0
            if vocab[w_i] in item:
                c1 += 1
                tmp += 1
            if vocab[w_j] in item:
                c2 += 1
                tmp += 1
            if tmp == 2:
                c3 += 1
        
        a = c3 / docs
        b = c1 / docs
        c = c2 / docs
        if b != 0 and c != 0:
            score -= np.log(a / b * c)
score /= len(top_words_per_topic)
print(score)

41.78657293197526
CPU times: user 10min 28s, sys: 46.4 ms, total: 10min 29s
Wall time: 10min 28s


In [42]:
t2d = pd.DataFrame(documentToTopic.T)

top_docs_per_topic = [
    (t2d.iloc[k][t2d.iloc[k] > 0.001].sort_values(ascending=False).index.values.tolist()[:100])
    for k in topics
]

a = np.asarray(top_docs_per_topic)
a = a.transpose()
np.savetxt("topic_document_rank.csv", a, delimiter=",")